In [2]:
#调包
from surprise import Dataset
from surprise import Reader
from surprise import BaselineOnly, KNNBasic, NormalPredictor
from surprise import accuracy
from surprise.model_selection import KFold



In [3]:

# 数据读取
path = 'C:/Users/Yan3/Desktop/dataset/ratings.csv'
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
data = Dataset.load_from_file(path, reader=reader)
train_set = data.build_full_trainset()

#### ALS优化算法
#### 原理:Step1，固定Y 优化X
#### Step2，固定X 优化Y
#### 重复Step1和2，直到X 和Y 收敛。每次固定一个矩阵，优化另一个矩阵，都是最小二乘问题

In [4]:

# ALS优化
bsl_options = {'method': 'als','n_epochs': 5,'reg_u': 12,'reg_i': 5}

#### Baseline算法
#### 基于统计的基准预测线打分.使用ALS进行优化
#### Step1，固定bu，优化bi
#### Step2，固定bi，优化bu


In [5]:
algo = BaselineOnly(bsl_options=bsl_options)

In [8]:
# 定义K折交叉验证迭代器，K=4
kf = KFold(n_splits=4)
for trainset, testset in kf.split(data):
    # 训练并预测
    algo.fit(trainset)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

uid = str(100)
iid = str(250)
# 输出uid对iid的预测结果
pred_baseline = algo.predict(uid, iid, r_ui=4, verbose=True)

Estimating biases using als...
RMSE: 0.8625
Estimating biases using als...
RMSE: 0.8634
Estimating biases using als...
RMSE: 0.8639
Estimating biases using als...
RMSE: 0.8611
user: 100        item: 250        r_ui = 4.00   est = 2.83   {'was_impossible': False}


#### Slopeone算法
#### SlopeOne是迄今为止非常简单且有效的一种完全基于统计的协同过滤算法，该算法的主要思想是通过当前用户打分的所有item与当前item i在同一用户下的得分偏差总和来估算当前用户对item i的打分。

In [9]:
from surprise import SlopeOne
# 使用SlopeOne算法
algo = SlopeOne()
# 定义K折交叉验证迭代器，K=4
kf = KFold(n_splits=4)
for trainset, testset in kf.split(data):
    algo.fit(train_set)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)
# 对指定用户和商品进行评分预测
uid = str(100) 
iid = str(250) 
pred_slope = algo.predict(uid, iid, r_ui=4, verbose=True)

RMSE: 0.7922
RMSE: 0.7940
RMSE: 0.7923
RMSE: 0.7919
user: 100        item: 250        r_ui = 4.00   est = 2.60   {'was_impossible': False}


#### SGD优化算法
#### 原理:基本思路是以随机方式遍历训练集中的数据，并给出每个已知评分的预测评分。用户和物品特征向量的调整就沿着评分误差越来越小的方向迭代进行，直到误差达到要求.

In [10]:
#SGD优化
bsl_options = {'method': 'sgd','n_epochs': 5}

#### KNN算法
#### kNN算法是从训练集中找到和新数据最接近的k条记录，然后根据他们的主要分类来决定新数据的类别。
#### 计算步骤如下：
####    1）算距离：给定测试对象，计算它与训练集中的每个对象的距离
####    2）找邻居：圈定距离最近的k个训练对象，作为测试对象的近邻
####    3）做分类：根据这k个近邻归属的主要类别，来对测试对象分类

In [11]:
sim_options = {'name': 'pearson_baseline'}
algo = KNNBasic(bsl_options=bsl_options, sim_options=sim_options)
# 定义K折交叉验证迭代器，K=4
kf = KFold(n_splits=4)
for trainset, testset in kf.split(data):
    algo.fit(train_set)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)
# 对指定用户和商品进行评分预测
uid = str(100) 
iid = str(250) 
pred_knn = algo.predict(uid, iid, r_ui=4, verbose=True)

Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.5705
Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.5725
Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.5736
Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.5719
user: 100        item: 250        r_ui = 4.00   est = 2.93   {'actual_k': 20, 'was_impossible': False}


#### Normalpredict算法
#### Normal Perdictor 认为用户对物品的评分是服从正态分布的，从而可以根据已有的评分的均值和方差 预测当前用户对其他物品评分的分数。


In [12]:

algo = NormalPredictor()
# 定义K折交叉验证迭代器，K=4
kf = KFold(n_splits=4)
for trainset, testset in kf.split(data):
    algo.fit(train_set)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)
# 对指定用户和商品进行评分预测
uid = str(100) 
iid = str(250) 
pred_norm = algo.predict(uid, iid, r_ui=4, verbose=True)

RMSE: 1.4330
RMSE: 1.4294
RMSE: 1.4313
RMSE: 1.4324
user: 100        item: 250        r_ui = 4.00   est = 2.78   {'was_impossible': False}


#### 从以上使用的4种算法的RMSE指标来看,对于这个movielens数据集来说,或许knn算法是一个不错的算法.
#### PS:本人的笔记本内存只有8G,所以kfold的k只设到4,否则,slopeone算法就让我的电脑卡死了...